In [36]:
from typing import Annotated
from dotenv import load_dotenv
load_dotenv()

from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [37]:
from langchain.chat_models import init_chat_model

In [38]:
llm = init_chat_model("google_genai:gemini-2.0-flash")

class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

In [39]:
message = {"role": "user", "content": "Who walked on the moon for the first time? Print only the name"}
# message = {"role": "user", "content": "What is the latest price of MSFT stock?"}
response = graph.invoke({"messages":[message]})

response["messages"]

[HumanMessage(content='Who walked on the moon for the first time? Print only the name', additional_kwargs={}, response_metadata={}, id='4cb9c6d6-7d75-4869-82f7-07f44398c1a7'),
 AIMessage(content='Neil Armstrong', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--21337791-00a1-4510-a2d1-bfcd33b196c7-0', usage_metadata={'input_tokens': 14, 'output_tokens': 3, 'total_tokens': 17, 'input_token_details': {'cache_read': 0}})]

In [40]:
state = None

while True:
    try:
        in_message = input("You: ")
        if in_message.lower() in {"quit", "exit"}:
            print("Bot: Goodbye!")
            break

        if state is None:
            state: State = {
                "messages": [{"role": "user", "content": in_message}]
            }
        else:
            state["messages"].append({"role": "user", "content": in_message})

        # Call your LangChain graph or runnable
        state = graph.invoke(state)

        # The last message might now be an AIMessage object
        last_message = state["messages"][-1]
        print("Bot:", last_message.content)

        # Add the follow-up message
        print("Bot: Let me know what else I can help you with.\n")

    except KeyboardInterrupt:
        print("\nBot: Interrupted. Exiting.")
        break


Bot: There are several people named Deepak Chaudhari. To give you the most relevant information, I need a little more context. Could you tell me:

*   **What field are you interested in?** (e.g., politics, business, sports, entertainment, academia)
*   **Where are you located (or where is this Deepak Chaudhari likely to be located)?** (e.g., India, US, UK)
*   **Do you have any other details?** (e.g., any specific organization they're associated with, a rough time period you're interested in)

In the meantime, here are a few possibilities:

*   **Deepak Chaudhari (Indian Politician):** There are multiple politicians and public figures named Deepak Chaudhari in India. They are often associated with local or regional politics.
*   **Deepak Chaudhari (Academics/Professionals):** There are professionals with that name in various fields like engineering, technology, business, and finance. Some may have academic affiliations.

Once you provide more information, I can give you a more specific